In [3]:
import pandas as pd
import numpy as np

import tensorflow as tf
from keras import backend as K
import keras.callbacks as callbacks
from keras import models
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.optimizers import SGD
from keras.datasets import mnist
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from keras.regularizers import L1L2
from keras.layers import BatchNormalization, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import log_loss
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import PolynomialFeatures
%matplotlib inline
from sklearn.preprocessing import OrdinalEncoder

import warnings
warnings.filterwarnings("ignore")
from xgboost import XGBClassifier
import os,boto3,requests,io,sys,re,time
from io import StringIO 
bucket_name = 'sftp-analytics-stg-transferbucket-a3e4afdvn23'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import lightgbm as lgb
pd.options.display.max_rows = 4000
import lightgbm as lgb


pd.set_option('display.float_format', lambda x: '%.5f' % x)


from numba import jit

@jit
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

def gini_lgb(preds, lgb_train):  
    actuals = np.array(lgb_train.get_label())   
    return 'gini', eval_gini(actuals, preds), True

def num_missing(x):
  return sum(x.isnull())

def impxgb(valores,variables):
    dictimp={variables[a]:valores[a] for a in range(0,len(variables)) }
    xgimp=sorted(list(dictimp.items()), key=lambda x: x[1],reverse=True)

    return xgimp


/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [33]:
query = 'select * from e_ba_mdl.rf_il_7'

In [34]:
client = boto3.client('athena')
queryStart = client.start_query_execution(
   QueryString = query,
    ResultConfiguration = {
            'OutputLocation':  's3://'+ bucket_name+'/data/ouput'
            }
        )
response = client.get_query_execution(QueryExecutionId = queryStart.get('QueryExecutionId'))

In [36]:
client = boto3.client('s3')
object_key = response.get('QueryExecution',{}).get('ResultConfiguration',{}).get('OutputLocation')
object_key = re.findall("a3e4afdvn23/(.*)",object_key)
object_key = object_key[0]
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
Body = csv_obj['Body']
csv_string = Body.read().decode('latin-1')
df = pd.read_csv(StringIO(csv_string),sep=",")
df.head()

,num_ce_m01,num_productos_m01,num_llamadas_m01,max_num_ce_u3m,prm_num_ce_u3m,max_num_acepta_u3m,prm_num_productos_u3m,max_num_llamadas_u3m,prm_num_llamadas_u3m,codmes,...,frec_camp_carga_il,sgt_cem,ctd_proac,ctd_cttefeproac,rng_pst,ctd_cttefetot,ctd_productosm01,rat_actpas,sexo,region
0,0,0,0,0,0.00000,0,0.00000,0,0.00000,201911,...,0,2,13,0,2,0,3.00000,1.64500,0,LIMA
1,1,1,1,3,2.00000,2,2.00000,31,20.33333,202003,...,0,1,16,2,2,2,5.00000,1.68900,0,SUR
2,1,1,2,1,1.00000,1,1.00000,5,3.50000,202003,...,0,1,7,0,2,0,5.00000,0.57900,0,LIMA
3,1,1,4,3,1.66667,1,1.00000,9,5.33333,202003,...,1,2,10,2,4,2,5.00000,729.33100,1,SUR
4,1,2,8,1,1.00000,1,1.50000,8,4.50000,202003,...,0,0,10,0,1,0,2.00000,nan,0,NORTE


In [96]:
df.shape
df.groupby(['codmes','target']).size()
list(df.columns)

(295448, 103)

codmes  target
201901  0         10883
        1          8045
201902  0         12169
        1          8836
201903  0         10439
        1         11202
201904  0         13789
        1          7430
201905  0         11468
        1         10662
201906  0         13409
        1         10245
201907  0         12084
        1          8737
201908  0          9948
        1          7714
201909  0         10506
        1          9409
201910  0         11841
        1          8846
201911  0         13776
        1          8572
201912  0         12345
        1          7761
202001  0          6654
        1          4924
202002  0         15156
        1          7529
202003  0          7574
        1          3495
dtype: int64

['num_ce_m01',
 'num_productos_m01',
 'num_llamadas_m01',
 'max_num_ce_u3m',
 'prm_num_ce_u3m',
 'max_num_acepta_u3m',
 'prm_num_productos_u3m',
 'max_num_llamadas_u3m',
 'prm_num_llamadas_u3m',
 'codmes',
 'key_value',
 'codunicocli',
 'mntcnsmo_1',
 'mntcnsmo_2',
 'mntcnsmo_3',
 'mntcnsmo_4',
 'mntcnsmo_5',
 'mntcnsmo_6',
 'target',
 'rec_trx_tc_amt_u10d',
 'rec_trx_cat_cmp_amt_u7d',
 'frc_trx_tc_amt_u10d',
 'rec_trx_tc_cnl_pos_amt_u7d',
 'frc_trx_tc_cnl_pos_amt_u7d',
 'prm_adj_trx_tc_amt_u10d',
 'max_trx_tc_amt_u10d',
 'max_trx_tc_cnl_pos_amt_u7d',
 'prm_trx_tc_cnl_pos_amt_u7d',
 'rec_trx_tc_cnl_bpi_amt_u7d',
 'prm_adj_sld_tcr_amt_u10d',
 'max_sld_tcr_amt_u10d',
 'prm_adj_sld_act_amt_u7d',
 'edad',
 'ptj_buro',
 'prm_usoep1tcallsf06m',
 'prm_sldvigibksf03m',
 'prm_sowe1ptcallsf06m',
 'prm_usoep1tcallsf06m_ingb',
 'sow_svep1tcrallsfm02_ingb',
 'tnd_usoep1tcallsfm01_ingb',
 'prm_sowe1ptcallsf06m_ingb',
 'tnd_lintcrallsfm01',
 'tnd_lintcribksfm01',
 'prm_swsvep1allsf12m',
 'tnd_lintcrr

In [23]:
pd.concat([df.isnull().sum()/df.shape[0], df.isnull().sum(),df.count(),df.nunique()], keys= ["%NULLS","COUNT_NULLS","NOT_NULL","Unique_Values"],axis=1).sort_values('%NULLS',ascending = False)

,%NULLS,COUNT_NULLS,NOT_NULL,Unique_Values
max_mescttefetot,0.50753,149948,145500,20
sldtot_prmtrxm06,0.50559,149375,146073,66907
rat_actpas,0.45875,135537,159911,62813
ctd_trx_tcr_m03,0.29540,87276,208172,156
ctd_trx_tcr_m02,0.25737,76040,219408,174
ctd_trx_tcr_m01,0.20967,61947,233501,187
incremento,0.19609,57935,237513,7277
ant_cli,0.16242,47988,247460,387
linea_nueva,0.13404,39603,255845,2029
linea_actual,0.13404,39601,255847,4489


In [41]:
df.groupby('region').size()
df.region.value_counts(dropna='False')

region
0    183178
1     33671
2     31714
3     19855
4     11030
5     16000
dtype: int64

0    183178
1     33671
2     31714
3     19855
5     16000
4     11030
Name: region, dtype: int64

In [37]:
df.region = df.region.fillna('OTROS') 
df.loc[df.region.isin(['INF. NO DISPONIBLE','OTROS']), 'region'] = 'OTROS' 
df.region = df.region.replace({'LIMA':0,'SUR':1,'NORTE':2,'CENTRO SUR':3,'CENTRO NORTE':4,'OTROS':5}) 

In [38]:
df.sit_lab.fillna('INDEPENDIENTE', inplace=True)
df.sit_lab = df.sit_lab.replace({'INDEPENDIENTE':0,'MIXTO':1,'DEPENDIENTE':2})
df.ptj_buro.fillna(0, inplace=True)
df.edad.fillna(df.edad.median(),inplace=True)


df.linea_actual = df.linea_actual.fillna(df.linea_actual.median(skipna=True))
df.linea_nueva = df.linea_nueva.fillna(df.linea_nueva.median(skipna=True))
df.incremento = df.incremento.fillna(df.incremento.median(skipna=True))
df = df.fillna(0)



In [39]:
ckt = df.loc[:,['codmes','codunicocli','key_value','target']]
dft = df.drop(['codmes','codunicocli','key_value','target'],axis=1)

scale = dft.drop([
    
    'cem',
    'ptj_buro',
    'ant_cli',
   
    'sit_lab',
    
    'flg_sldacttcr',
    'prm_campadqtcrtlv06m',
    
    
    'max_mescttefetot',
   'edad'
],axis=1)
scale = scale.mask((scale < scale.quantile(0.05)), scale.quantile(0.05), axis=1)  
scale = scale.mask((scale > scale.quantile(0.95)), scale.quantile(0.95), axis=1)   
scale = pd.concat([dft[dft.columns.difference(scale.columns)],scale],axis=1)
scaled_features = StandardScaler().fit_transform(scale)
scaled_features_df = pd.DataFrame(scaled_features, index=scale.index, columns=scale.columns)

dft3= scaled_features_df



 
# dft3.loc[:,'prm_consumo_1'] = dft3.mntcnsmo_1/dft3.ctd_trx_tcr_m01
# dft3.loc[:,'prm_consumo_2']= dft3.mntcnsmo_2/dft3.ctd_trx_tcr_m02
# dft3.loc[:,'prm_consumo_3']= dft3.mntcnsmo_3/dft3.ctd_trx_tcr_m03
# dft3.loc[:,'prm_consumo_6']= (((dft3.mntcnsmo_1+ dft3.mntcnsmo_2 + dft3.mntcnsmo_3 + dft3.mntcnsmo_4 +dft3.mntcnsmo_5 +  dft3.mntcnsmo_6
#                      )/6)/(dft3.prm_frc_trx_06m))

# dfp = pd.concat([
# dft3.rec_camptottlv06m,
# dft3.frc_camptottlv06m ,
# dft3.prm_usoep1tcallsf06m_ingb,
# dft3.tnd_lintcribksfm01_ingb,
# dft3.prm_usotcrrstsf06m_ingb,
# dft3.incremento,
# dft3.max_mescttefetot],axis=1)
# poly_transformer = PolynomialFeatures(degree = 2)
# poly_transformer.fit(dfp)

# df_p = poly_transformer.transform(dfp)

# feature_name = poly_transformer.get_feature_names(input_features = ['rec_camptottlv06m',
#                                                                     'frc_camptottlv06m',
#                                                                     'prm_usoep1tcallsf06m_ingb',
#                                                                     'tnd_lintcribksfm01_ingb',
#                                                                     'prm_usotcrrstsf06m_ingb',
#                                                                     'incremento'
#                                                                    ,'max_mescttefetot'])
 

# poly_features = pd.DataFrame(df_p, columns = feature_name,index=dft3.index.copy())
# poly_features.drop(['rec_camptottlv06m',
#                     'frc_camptottlv06m',
#                     'prm_usoep1tcallsf06m_ingb',
#                     'tnd_lintcribksfm01_ingb',
#                     'prm_usotcrrstsf06m_ingb',
#                     'incremento'
#                     ,'max_mescttefetot','1'],axis=1,inplace=True)

# dft3 = pd.concat([dft3,poly_features],axis=1)




dft6 = pd.concat([dft3,ckt],axis=1)

In [100]:
dft6.head()

,ant_cli,cem,edad,flg_sldacttcr,max_mescttefetot,prm_campadqtcrtlv06m,ptj_buro,sit_lab,num_ce_m01,num_productos_m01,...,rng_pst,ctd_cttefetot,ctd_productosm01,rat_actpas,sexo,region,codmes,codunicocli,key_value,target
0,-0.61168,-0.47230,0.19090,0.28652,1.01445,-0.36319,-0.20868,0.39906,-0.72101,-1.12312,...,0.37257,0.04860,-0.41319,-0.30328,-0.96474,2.73264,201902,14881761,7876FD1ED47EC51A078AAC49DCAD5719C34BD6ACED6CE3...,0
1,0.42374,3.97172,0.70643,0.28652,1.01448,-0.36319,0.33520,-0.90971,1.51732,0.21685,...,-1.41371,0.88056,-0.41319,-0.30328,1.03655,-0.64130,201902,12347303,789CFC4B119157326F3AEEAB071B48466E271C35964F25...,0
2,1.49101,1.31634,1.82340,0.28652,-0.98506,-0.36319,0.27079,-0.90971,-0.72101,0.21685,...,-1.41371,-0.78335,-0.41319,-0.30328,1.03655,-0.64130,201902,9625316,7902877C1E0B0FDCD55C5C3050BCB3C8AE9236F426DEEF...,0
3,-1.04177,-0.47230,-0.66831,0.28652,1.01448,-0.36319,0.34951,-0.90971,1.51732,1.55682,...,0.37257,0.04860,0.24070,-0.30284,1.03655,0.03349,201902,14545418,791492837629CEBDB12AB48145A45E35574436B7D08265...,0
4,-0.21344,1.13527,-0.58239,0.28652,1.01448,-0.36319,0.43539,-0.90971,1.51732,1.55682,...,0.37257,1.71251,-1.06708,-0.30328,-0.96474,2.73264,201902,13852841,7A0BEDD112DB68C708ABA0DB14F9CD0E8051AEF2FF7E7D...,0


In [65]:
dft6 = dft6 [[
'frec_camp_carga_il', 
'total_camp_carga_il_u6m', 
'recencia_camp_carga_il', 
'tnd_lintcribksfm01_ingb', 
'ctd_proac', 
'prm_frc_trx_06m',
'prm_num_productos_u3m', 
'max_num_ce_u3m',
'sit_lab',
'max_num_acepta_u3m',
'tnd_lintcrallsfm01_ingb',
'linea_nueva',
'dist_ctd_ent_tot_m01',
'rng_pst',
'prm_num_ce_u3m',
'linea_actual',
'edad',
'rat_actpas',
'prm_usoep1tcallsf06m_ingb',
'num_ce_m01', 
'dsv_salvigprmtcr06m',
'tnd_lintcribksfm01', 
'region', 
'incremento', 
'ptj_buro',
'prm_usotcrrstsf06m_ingb', 
'prm_sldvigibksf03m', 
'sldtot_prmtrxm06',
'prm_usotcribksf03m_ingb',
'ant_cli',
'max_mescttefetot', 
'mntcnsmo_1']]


In [109]:
dft6 = dft6 [[

'total_camp_carga_il_u6m',
'frec_camp_carga_il',
'recencia_camp_carga_il',
'total_camp_carga_tlv_il_u6m',
'frec_camp_carga_tlv_il',
'max_num_acepta_u3m',
'ctd_proac',
'prm_num_productos_u3m',
'frc_camptottlv06m',
'num_productos_m01',
'prm_usoep1tcallsf06m_ingb',
'prm_num_ce_u3m',
'sgt_cem',
'incremento',
'tnd_lintcribksfm01_ingb',
'prm_usotcrrstsf06m_ingb',
'sit_lab',
'num_ce_m01',
'max_usotcrallsf03m_ingb',
'rec_camptottlv06m',
'flg_sldacttcr',
'rng_pst',
'prm_usotcrallsf03m_ingb',
'max_num_ce_u3m',
'linea_nueva',
'tnd_lintcrallsfm01_ingb',
'prm_usotcribksf03m_ingb',
'max_num_llamadas_u3m',
'prm_frc_trx_06m',
'prm_usotcribksf06m_ingb',
'prm_sowe1ptcallsf06m_ingb',
'prm_num_llamadas_u3m',
'prm_usoep1tcallsf06m',
'max_mescttefetot',
'num_llamadas_m01',
'uso_tc_rccibk_m01_ingb',
'sow_svep1tcrallsfm02_ingb',
'ctd_trx_tcr_m02',
'ctd_cttefeproac',
'prm_usotcrrstsf06m',
'ctd_trx_tcr_m01',
'tnd_lintcribksfm01',
'linea_actual',
'uso_tc_rccsf_m01',
'prm_usotcrallsf03m',
'prm_swsvep1allsf12m_ingb',
'ctd_trx_tcr_m03',
'ctd_cttefetot',
'dist_ctd_ent_tot_m01',
'max_swsvep1allsf03m_ingb',
'prm_usotcribksf03m',
'prm_campadqtcrtlv06m',
'prm_swsvep1allsf03m_ingb',
'tnd_lintcrallsfm01',
'ptj_buro',
'tnd_lintcrrstsfm01_ingb',
'prm_usotcribksf06m',
'mntcnsmo_6',
'prm_sowe1ptcallsf06m',
'prm_ingnotfnctcr06m_ingb',
'ctd_productosm01',
'flg_pa',
'cem',
'mntcnsmo_4',
'prm_swsvep1allsf03m',
'mntcnsmo_3',
'mntcnsmo_2',
'dsv_salvigprmtcr06m',
'mntcnsmo_5',
'prm_ingnotfnc06m_ingb',
'prm_swsvep1allsf12m',
'frc_trx_tc_amt_u10d',
'max_swsvep1allsf03m',
'mntcnsmo_1',
'tnd_usoep1tcallsfm01_ingb',
'region',
'frc_trx_tc_cnl_pos_amt_u7d',
'sldtot_prmtcrm03',
'lintot_tc_rccsf_03m_ingb',
'prm_adj_sld_act_amt_u7d',
'prm_sldvigibksf03m',
'edad',
'rec_trx_tc_amt_u10d',
'tnd_usotcribksfm01_ingb',
'prm_adj_sld_tcr_amt_u10d',
'max_sld_tcr_amt_u10d',
'sldvig_prmtcrm06',
'tnd_lintcrrstsfm01' 
]]

In [110]:
dft6 = pd.concat([dft6,ckt],axis=1)

In [111]:
dft6.shape
list(dft6.columns)

(295448, 92)

['total_camp_carga_il_u6m',
 'frec_camp_carga_il',
 'recencia_camp_carga_il',
 'total_camp_carga_tlv_il_u6m',
 'frec_camp_carga_tlv_il',
 'max_num_acepta_u3m',
 'ctd_proac',
 'prm_num_productos_u3m',
 'frc_camptottlv06m',
 'num_productos_m01',
 'prm_usoep1tcallsf06m_ingb',
 'prm_num_ce_u3m',
 'sgt_cem',
 'incremento',
 'tnd_lintcribksfm01_ingb',
 'prm_usotcrrstsf06m_ingb',
 'sit_lab',
 'num_ce_m01',
 'max_usotcrallsf03m_ingb',
 'rec_camptottlv06m',
 'flg_sldacttcr',
 'rng_pst',
 'prm_usotcrallsf03m_ingb',
 'max_num_ce_u3m',
 'linea_nueva',
 'tnd_lintcrallsfm01_ingb',
 'prm_usotcribksf03m_ingb',
 'max_num_llamadas_u3m',
 'prm_frc_trx_06m',
 'prm_usotcribksf06m_ingb',
 'prm_sowe1ptcallsf06m_ingb',
 'prm_num_llamadas_u3m',
 'prm_usoep1tcallsf06m',
 'max_mescttefetot',
 'num_llamadas_m01',
 'uso_tc_rccibk_m01_ingb',
 'sow_svep1tcrallsfm02_ingb',
 'ctd_trx_tcr_m02',
 'ctd_cttefeproac',
 'prm_usotcrrstsf06m',
 'ctd_trx_tcr_m01',
 'tnd_lintcribksfm01',
 'linea_actual',
 'uso_tc_rccsf_m01',
 '

In [108]:
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier(n_estimators=500, random_state=42, max_leaf_nodes=16)
model.fit(dft6.iloc[:,:-4],dft6.target)
feature_importances_extra = pd.Series(model.feature_importances_, dft6.iloc[:,:-4].columns).sort_values(ascending=False)
feature_importances_extra

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=16,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

total_camp_carga_il_u6m       0.16924
frec_camp_carga_il            0.14779
recencia_camp_carga_il        0.06443
total_camp_carga_tlv_il_u6m   0.05811
frec_camp_carga_tlv_il        0.04521
max_num_acepta_u3m            0.04123
ctd_proac                     0.04085
prm_num_productos_u3m         0.03854
frc_camptottlv06m             0.02864
num_productos_m01             0.02807
prm_usoep1tcallsf06m_ingb     0.02053
prm_num_ce_u3m                0.02035
sgt_cem                       0.01743
incremento                    0.01740
tnd_lintcribksfm01_ingb       0.01670
prm_usotcrrstsf06m_ingb       0.01541
sit_lab                       0.01439
num_ce_m01                    0.01260
max_usotcrallsf03m_ingb       0.01234
rec_camptottlv06m             0.01159
flg_sldacttcr                 0.01138
rng_pst                       0.00969
prm_usotcrallsf03m_ingb       0.00962
max_num_ce_u3m                0.00961
linea_nueva                   0.00942
tnd_lintcrallsfm01_ingb       0.00794
prm_usotcrib

In [40]:
train = dft6.loc[dft6["codmes"]<202001]
test = dft6.loc[dft6["codmes"]==202001]

# val_201911 = dft6.loc[dft6["codmes"]==201911]
# val_201912 = dft6.loc[dft6["codmes"]==201912]
# val_202001 = dft6.loc[dft6["codmes"]==202001]
val_202002 = dft6.loc[dft6["codmes"]==202002]


X_train =  train.iloc[:,:-4]
y_train =  train.target

X_test =  test.iloc[:,:-4]
y_test =  test.target

# X_val_201911 =  val_201911.iloc[:,:-4]
# y_val_201911 =  val_201911.target

# X_val_201912 =  val_201912.iloc[:,:-4]
# y_val_201912 =  val_201912.target
j
# X_val_202001 =  val_202001.iloc[:,:-4]
# y_val_202001 =  val_202001.target

X_val_202002 =  val_202002.iloc[:,:-4]
y_val_202002 =  val_202002.target

dtrain =       lgb.Dataset(pd.DataFrame(X_train), pd.DataFrame(y_train))
dtest =        lgb.Dataset(pd.DataFrame(X_test), pd.DataFrame(y_test), reference=dtrain)


In [41]:
params = {
          'boosting_type': 'gbdt',
          'max_depth' : -1,
          'objective': 'binary',
    'is_unbalance': False,
              'nthread': 3, 
          #'num_leaves': 41,
          'learning_rate': 0.05,
          'max_bin': 50,
          'subsample_for_bin': 200,
          'subsample': 1,
          'subsample_freq': 1,
          'colsample_bytree': 0.8,
          'reg_alpha': 5,
          'reg_lambda': 10,
          #'min_split_gain': 0.5,         
          'min_child_weight': 1,
          'min_child_samples': 5,
          'scale_pos_weight': 1,
          #'min_sum_hessian_in_leaf': 226,
          'num_class' : 1,
          'metric' : 'auc',
         'verbose': 1,
    'bagging_fraction': 0.5540557845037332,
 'bagging_freq': 1,
 'feature_fraction': 0.5068213222676954,
 'lambda_l1': 1,
 'lambda_l2': 0.6088814046649453,
 'max_depth': 7,
 'min_data_in_leaf': 239,
 'min_gain_to_split': 1,
 'min_sum_hessian_in_leaf': 225,
 'num_leaves': 45,
 'weight': 234.38737874771672}

In [103]:
%%time
#new
gbm = lgb.train(params,
            dtrain,
            verbose_eval = 100,
            num_boost_round=10000,
            valid_sets=[dtrain,dtest],
            early_stopping_rounds=300,
            feval = gini_lgb)

Training until validation scores don't improve for 300 rounds
[100]	training's auc: 0.74887	training's gini: 0.497739	valid_1's auc: 0.757906	valid_1's gini: 0.515811
[200]	training's auc: 0.759708	training's gini: 0.519417	valid_1's auc: 0.761717	valid_1's gini: 0.523433
[300]	training's auc: 0.766947	training's gini: 0.533894	valid_1's auc: 0.762959	valid_1's gini: 0.525917
[400]	training's auc: 0.772721	training's gini: 0.545441	valid_1's auc: 0.763063	valid_1's gini: 0.526125
[500]	training's auc: 0.777831	training's gini: 0.555663	valid_1's auc: 0.762854	valid_1's gini: 0.525707
[600]	training's auc: 0.782395	training's gini: 0.56479	valid_1's auc: 0.763133	valid_1's gini: 0.526265
[700]	training's auc: 0.786515	training's gini: 0.573031	valid_1's auc: 0.762534	valid_1's gini: 0.525067
Early stopping, best iteration is:
[469]	training's auc: 0.776336	training's gini: 0.552672	valid_1's auc: 0.763295	valid_1's gini: 0.526589
CPU times: user 2min 23s, sys: 299 ms, total: 2min 23s
Wa

In [104]:
filename = 'lgbm_final_acept.sav'
pickle.dump(gbm, open(filename, 'wb'))

In [42]:
print('Acept_val_202002')
y_pred_2002 = gbm.predict(X_val_202002, ntree_limit=gbm.best_iteration)
roc_auc_score(y_val_202002, y_pred_2002)*2-1

Acept_val_202002


0.5699675865793639

In [16]:
gbm = pd.read_pickle('lgbm_final_acept.sav')

In [106]:
gbm.feature_name()

['ant_cli',
 'cem',
 'edad',
 'flg_sldacttcr',
 'max_mescttefetot',
 'prm_campadqtcrtlv06m',
 'ptj_buro',
 'sit_lab',
 'num_ce_m01',
 'num_productos_m01',
 'num_llamadas_m01',
 'max_num_ce_u3m',
 'prm_num_ce_u3m',
 'max_num_acepta_u3m',
 'prm_num_productos_u3m',
 'max_num_llamadas_u3m',
 'prm_num_llamadas_u3m',
 'mntcnsmo_1',
 'mntcnsmo_2',
 'mntcnsmo_3',
 'mntcnsmo_4',
 'mntcnsmo_5',
 'mntcnsmo_6',
 'rec_trx_tc_amt_u10d',
 'rec_trx_cat_cmp_amt_u7d',
 'frc_trx_tc_amt_u10d',
 'rec_trx_tc_cnl_pos_amt_u7d',
 'frc_trx_tc_cnl_pos_amt_u7d',
 'prm_adj_trx_tc_amt_u10d',
 'max_trx_tc_amt_u10d',
 'max_trx_tc_cnl_pos_amt_u7d',
 'prm_trx_tc_cnl_pos_amt_u7d',
 'rec_trx_tc_cnl_bpi_amt_u7d',
 'prm_adj_sld_tcr_amt_u10d',
 'max_sld_tcr_amt_u10d',
 'prm_adj_sld_act_amt_u7d',
 'prm_usoep1tcallsf06m',
 'prm_sldvigibksf03m',
 'prm_sowe1ptcallsf06m',
 'prm_usoep1tcallsf06m_ingb',
 'sow_svep1tcrallsfm02_ingb',
 'tnd_usoep1tcallsfm01_ingb',
 'prm_sowe1ptcallsf06m_ingb',
 'tnd_lintcrallsfm01',
 'tnd_lintcribks

In [18]:
importancia=gbm.feature_importance(importance_type='gain')
ixg=impxgb(importancia,X_train.columns)
pd.Series(ixg)

0              (frec_camp_carga_il, 65194.623300909996)
1          (total_camp_carga_il_u6m, 27189.24461734295)
2          (recencia_camp_carga_il, 19305.714750647545)
3          (tnd_lintcribksfm01_ingb, 17773.92824523896)
4                       (ctd_proac, 12298.410036802292)
5                   (prm_frc_trx_06m, 8808.71262896061)
6                      (linea_nueva, 8484.417713224888)
7                   (max_num_ce_u3m, 8397.061619848013)
8                          (sit_lab, 7840.612907886505)
9            (prm_num_productos_u3m, 7799.076124787331)
10              (max_num_acepta_u3m, 7636.009835243225)
11                    (linea_actual, 7583.807394891977)
12         (tnd_lintcrallsfm01_ingb, 7558.073608100414)
13             (dist_ctd_ent_tot_m01, 7136.08930182457)
14                            (edad, 6485.279686778784)
15             (dsv_salvigprmtcr06m, 6152.919001847506)
16                         (rng_pst, 5907.606671571732)
17                  (prm_num_ce_u3m, 5727.606276

In [22]:
dft6["score_acept"]  = gbm.predict(pd.DataFrame(dft6.iloc[:,:-4]), ntree_limit=gbm.best_iteration)

In [24]:
val_202002.shape

(22685, 77)

In [23]:
# val_201911 = dft6.loc[dft6["codmes"]==201911]
# val_201912 = dft6.loc[dft6["codmes"]==201912]
# val_202001 = dft6.loc[dft6["codmes"]==202001]
val_202002 = dft6.loc[dft6["codmes"]==202002]

print("Deciles Aceptacion")

# print('201911')
# val_201911['decil'] = (pd.qcut(val_201911['score_acept'], 10, labels=False)-10)*-1
# val_201911[['target', 'decil']].groupby('decil').agg(['count', 'sum', 'mean'])

# print('201912')
# val_201912['decil'] = (pd.qcut(val_201912['score_acept'], 10, labels=False)-10)*-1
# val_201912[['target', 'decil']].groupby('decil').agg(['count', 'sum', 'mean'])

# print('202001')
# val_202001['decil'] = (pd.qcut(val_202001['score_acept'], 10, labels=False)-10)*-1
# val_202001[['target', 'decil']].groupby('decil').agg(['count', 'sum', 'mean'])

print('202002')
val_202002['decil'] = (pd.qcut(val_202002['score_acept'], 10, labels=False)-10)*-1
val_202002[['target', 'decil']].groupby('decil').agg(['count', 'sum', 'mean'])



Deciles Aceptacion
202002


target              
       count   sum    mean
decil                     
1       2269  1398 0.61613
2       2268  1299 0.57275
3       2269  1017 0.44822
4       2268   910 0.40123
5       2268   823 0.36287
6       2269   660 0.29088
7       2268   527 0.23236
8       2269   419 0.18466
9       2268   290 0.12787
10      2269   186 0.08197

#### Scoreo

In [4]:
del client
del queryStart
del response
del object_key
del csv_obj
del Body
del csv_string

NameError: name 'client' is not defined

In [5]:
query = 'select * from e_ba_mdl.rf_il_backtest_8'

In [6]:
client = boto3.client('athena')
queryStart = client.start_query_execution(
   QueryString = query,
    ResultConfiguration = {
            'OutputLocation':  's3://'+ bucket_name+'/data/ouput'
            }
        )
response = client.get_query_execution(QueryExecutionId = queryStart.get('QueryExecutionId'))

In [7]:
client = boto3.client('s3')
object_key = response.get('QueryExecution',{}).get('ResultConfiguration',{}).get('OutputLocation')
object_key = re.findall("a3e4afdvn23/(.*)",object_key)
object_key = object_key[0]
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
Body = csv_obj['Body']
csv_string = Body.read().decode('latin-1')
df = pd.read_csv(StringIO(csv_string),sep=",")
df.head()

,num_ce_m01,num_productos_m01,num_llamadas_m01,max_num_ce_u3m,prm_num_ce_u3m,max_num_acepta_u3m,prm_num_productos_u3m,max_num_llamadas_u3m,prm_num_llamadas_u3m,codmes,...,ctd_proac,ctd_cttefeproac,rng_pst,ctd_cttefetot,ctd_productosm01,rat_actpas,sexo,region,prob_value,grp_camptottlv06m
0,0,0,0,2,1.50000,1,1.50000,5,3.00000,202005,...,18,3,2,3,2.00000,nan,1,CENTRO SUR,0.59217,G5
1,0,0,0,0,0.00000,0,0.00000,0,0.00000,202005,...,9,0,2,0,2.00000,0.00000,0,LIMA,0.25953,G1
2,0,0,0,1,1.00000,1,1.50000,4,3.50000,202005,...,3,0,2,0,3.00000,0.32300,0,LIMA,0.37748,G3
3,0,0,0,0,0.00000,0,1.00000,3,3.00000,202005,...,12,0,2,0,4.00000,0.44000,0,LIMA,0.21734,G3
4,0,0,0,0,0.00000,0,0.00000,0,0.00000,202005,...,18,0,2,0,1.00000,nan,0,LIMA,0.44680,G5


In [8]:
df.groupby('codmes').size()

codmes
202005    73653
dtype: int64

In [9]:
df.region = df.region.fillna('OTROS') 
df.loc[df.region.isin(['INF. NO DISPONIBLE','OTROS']), 'region'] = 'OTROS' 
df.region = df.region.replace({'LIMA':0,'SUR':1,'NORTE':2,'CENTRO SUR':3,'CENTRO NORTE':4,'OTROS':5}) 

In [10]:
df.sit_lab.fillna('INDEPENDIENTE', inplace=True)
df.sit_lab = df.sit_lab.replace({'INDEPENDIENTE':0,'MIXTO':1,'DEPENDIENTE':2})
df.ptj_buro.fillna(0, inplace=True)
df.edad.fillna(df.edad.median(),inplace=True)


df.linea_actual = df.linea_actual.fillna(df.linea_actual.median(skipna=True))
df.linea_nueva = df.linea_nueva.fillna(df.linea_nueva.median(skipna=True))
df.incremento = df.incremento.fillna(df.incremento.median(skipna=True))
df = df.fillna(0)


In [11]:
ckt = df.loc[:,['codmes','codunicocli','key_value','grp_camptottlv06m','prob_value']]
dft = df.drop(['codmes','codunicocli','key_value','grp_camptottlv06m','prob_value'],axis=1)

scale = dft.drop([
    
    'cem',
    'ptj_buro',
    'ant_cli',
   
    'sit_lab',
    
    'flg_sldacttcr',
    'prm_campadqtcrtlv06m',
    
    
    'max_mescttefetot',
   'edad'
],axis=1)
scale = scale.mask((scale < scale.quantile(0.05)), scale.quantile(0.05), axis=1)  
scale = scale.mask((scale > scale.quantile(0.95)), scale.quantile(0.95), axis=1)   
scale = pd.concat([dft[dft.columns.difference(scale.columns)],scale],axis=1)
scaled_features = StandardScaler().fit_transform(scale)
scaled_features_df = pd.DataFrame(scaled_features, index=scale.index, columns=scale.columns)

dft3= scaled_features_df



 
# dft3.loc[:,'prm_consumo_1'] = dft3.mntcnsmo_1/dft3.ctd_trx_tcr_m01
# dft3.loc[:,'prm_consumo_2']= dft3.mntcnsmo_2/dft3.ctd_trx_tcr_m02
# dft3.loc[:,'prm_consumo_3']= dft3.mntcnsmo_3/dft3.ctd_trx_tcr_m03
# dft3.loc[:,'prm_consumo_6']= (((dft3.mntcnsmo_1+ dft3.mntcnsmo_2 + dft3.mntcnsmo_3 + dft3.mntcnsmo_4 +dft3.mntcnsmo_5 +  dft3.mntcnsmo_6
#                      )/6)/(dft3.prm_frc_trx_06m))

# dfp = pd.concat([
# dft3.rec_camptottlv06m,
# dft3.frc_camptottlv06m ,
# dft3.prm_usoep1tcallsf06m_ingb,
# dft3.tnd_lintcribksfm01_ingb,
# dft3.prm_usotcrrstsf06m_ingb,
# dft3.incremento,
# dft3.max_mescttefetot],axis=1)
# poly_transformer = PolynomialFeatures(degree = 2)
# poly_transformer.fit(dfp)

# df_p = poly_transformer.transform(dfp)

# feature_name = poly_transformer.get_feature_names(input_features = ['rec_camptottlv06m',
#                                                                     'frc_camptottlv06m',
#                                                                     'prm_usoep1tcallsf06m_ingb',
#                                                                     'tnd_lintcribksfm01_ingb',
#                                                                     'prm_usotcrrstsf06m_ingb',
#                                                                     'incremento'
#                                                                    ,'max_mescttefetot'])
 

# poly_features = pd.DataFrame(df_p, columns = feature_name,index=dft3.index.copy())
# poly_features.drop(['rec_camptottlv06m',
#                     'frc_camptottlv06m',
#                     'prm_usoep1tcallsf06m_ingb',
#                     'tnd_lintcribksfm01_ingb',
#                     'prm_usotcrrstsf06m_ingb',
#                     'incremento'
#                     ,'max_mescttefetot','1'],axis=1,inplace=True)

# dft3 = pd.concat([dft3,poly_features],axis=1)




#dft6 = pd.concat([dft3,ckt],axis=1)

In [12]:
gbm = pd.read_pickle('lgbm_final_acept.sav')

In [13]:
dft3["score_acept"]  = gbm.predict(dft3, ntree_limit=gbm.best_iteration)
dft6 = pd.concat([dft3,ckt],axis=1)

In [14]:
dft6.shape
dft6.head()
dft6.tail()
dft6.codmes.head()

(73653, 105)

,ant_cli,cem,edad,flg_sldacttcr,max_mescttefetot,prm_campadqtcrtlv06m,ptj_buro,sit_lab,num_ce_m01,num_productos_m01,...,ctd_productosm01,rat_actpas,sexo,region,score_acept,codmes,codunicocli,key_value,grp_camptottlv06m,prob_value
0,0.45188,-0.04028,1.87103,0.64645,1.69921,-0.31146,0.28563,-1.25184,0.00000,0.00000,...,0.04693,-0.30608,1.01242,1.59068,0.67321,202005,12105584,74B8CB2E5FFD49169E218A689E376355F7352E409FAB81...,G5,0.59217
1,-0.95057,-0.51771,-1.49724,0.64645,-0.58861,-0.31146,0.10500,-1.25184,0.00000,0.00000,...,0.04693,-0.30608,-0.98773,-0.71110,0.66064,202005,16128311,C42B3442AE1A664966C243923F907103D538BEFC246801...,G1,0.25953
2,0.98482,-0.37291,1.69830,0.64645,-0.58861,3.21070,0.33455,-1.25184,0.00000,0.00000,...,0.68580,-0.26456,-0.98773,-0.71110,0.66575,202005,10275194,3752976A6A9CA0DBA1D8D60FF4E7FAEBDBE0B9FE1040F2...,G3,0.37748
3,-0.23532,-0.28851,-0.63358,0.64645,-0.58861,-0.31146,0.34960,-1.25184,0.00000,0.00000,...,1.32468,-0.24953,-0.98773,-0.71110,0.51378,202005,11830250,37702C98841554C9D548B6ECADEE0A54B6764739E71261...,G3,0.21734
4,0.47993,-0.51771,-0.02902,0.64645,-0.58861,-0.31146,0.21413,0.05229,0.00000,0.00000,...,-0.59195,-0.30608,-0.98773,-0.71110,0.53807,202005,12001432,4AB4B19CC5C864337E9C5295C73E4C0C06F31303E2B411...,G5,0.44680


,ant_cli,cem,edad,flg_sldacttcr,max_mescttefetot,prm_campadqtcrtlv06m,ptj_buro,sit_lab,num_ce_m01,num_productos_m01,...,ctd_productosm01,rat_actpas,sexo,region,score_acept,codmes,codunicocli,key_value,grp_camptottlv06m,prob_value
73648,0.52201,-0.51771,0.31644,0.64645,1.69920,-0.31146,0.31950,1.35641,0.00000,0.00000,...,0.68580,3.99250,1.01242,-0.71110,0.15916,202005,8305719,D5ED115A02AE800BFA9038924268ED3EB862E0F9107F0D...,G4,0.50651
73649,0.99884,0.81447,-0.28812,-1.54690,-0.58861,3.21070,0.37594,-1.25184,0.00000,0.00000,...,-1.23082,-0.30608,-0.98773,0.82342,0.63816,202005,10240800,DA5EA2C337A63C44EE66C9EC84B9C8A81CAC559B1A939E...,G5,0.09102
73650,-0.86642,-0.51771,-1.49724,0.64645,-0.58861,-0.31146,0.33078,0.05229,0.00000,0.00000,...,-1.23082,-0.30608,1.01242,-0.71110,0.61423,202005,15835975,E2C9E109964636B6E04DF838FE48559DAA17AA42ADB709...,G1,0.04672
73651,-0.81033,-0.51771,-1.41088,0.64645,1.69921,-0.31146,0.23671,0.05229,0.00000,0.00000,...,0.04693,-0.13025,-0.98773,-0.71110,0.33202,202005,15673160,E415A7D36FB19A1E494DF51A9F65AB4A1A9D5CB8107EFA...,G4,0.56559
73652,0.29761,-0.10151,-0.97905,0.64645,1.69817,-0.31146,0.33455,1.35641,0.00000,0.00000,...,1.32468,-0.27176,1.01242,-0.71110,0.44562,202005,12446297,F6F69DCACC72E76035723D3758CE58F12226DFD3F57BCF...,G5,0.23793


0    202005
1    202005
2    202005
3    202005
4    202005
Name: codmes, dtype: int64

In [15]:
dft7 = pd.concat([dft6.loc[:,['codmes', 'key_value' ,'score_acept','prob_value', 'grp_camptottlv06m']], df.loc[:,['linea_nueva','incremento']]], axis=1)


In [16]:
dft7.to_csv('mayo_scored.csv')

In [30]:
dft7.head()

,codmes,key_value,score_acept,prob_value,grp_camptottlv06m,linea_nueva,incremento
0,202002,BFA9EB1B091E57B0D0FE0CE64FC5974FF49309E1B8ADC3...,0.06785,0.52296,G4,67300.00000,23600.00000
1,202002,C02E4BC1D6E67C991B3025BC773889002F35C52D9F6E99...,0.58248,0.17499,G4,19900.00000,14700.00000
2,202002,200562DCAEBD322664460A553F67911505ABA916003FE9...,0.41705,0.17180,G5,33200.00000,2500.00000
3,202002,20181822F05AF44C4342D424C20F54D3E3D96E854A6FEA...,0.15013,0.61351,G5,55000.00000,4600.00000
4,202002,201D7C463BB79B3902176D3B2EB154ECBF7E15CEFF336E...,0.20994,0.11730,G2,10200.00000,8700.00000


In [124]:
def bayes_parameter_opt_lgb_auc(X, y, init_round=15, opt_round=25, n_folds=5, random_seed=6, 
                            n_estimators=10000, learning_rate=0.05, output_process=False):
    # prepare data
    train_data = lgb.Dataset(data=X, label=y, feature_name = list(X))
    # parameters
    def lgb_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2,
                 min_gain_to_split, min_sum_hessian_in_leaf, bagging_freq, min_data_in_leaf, weight):
        params = {'application':'binary','num_iterations': n_estimators,
                  'learning_rate':learning_rate,
                  'early_stopping_round':100, 'metric':'auc'}
        
        params["num_leaves"] = int(round(num_leaves, 0))
        params['feature_fraction'] = round(max(min(feature_fraction, 1), 0),4)
        params['bagging_fraction'] = round(max(min(bagging_fraction, 1), 0),4)
        params['max_depth'] = int(round(max_depth, 0))
        params['lambda_l1'] = round(max(lambda_l1, 0),4)
        params['lambda_l2'] = round(max(lambda_l2, 0),4)
        params['bagging_freq'] = int(round(bagging_freq,0))
        params['min_gain_to_split'] = round(min_gain_to_split,4)
        params['min_sum_hessian_in_leaf'] = round(min_sum_hessian_in_leaf,4)
        params['min_data_in_leaf'] = int(round(min_data_in_leaf, 0))
        params['weight'] = round(weight,4)
        cv_result = lgb.cv(params, 
                           train_data, 
                           nfold=n_folds, 
                           seed=random_seed, 
                           stratified=True, 
                           verbose_eval =200,
                           metrics=['auc'],
                           feval=gini_lgb)
        #return max(cv_result['auc-mean'])
        return (cv_result['auc-mean'][-1])
    # range 
    lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (22, 48),
                                            'feature_fraction': (0.5, 0.7),
                                            'bagging_fraction': (0.5, 0.7),
                                            'max_depth': (7, 9),
                                            'lambda_l1': (0.1, 2),
                                            'lambda_l2': (0.1, 2),
                                            'bagging_freq': (1, 3),
                                            'min_gain_to_split': (1, 250),
                                            'min_sum_hessian_in_leaf': (1, 250),
                                            'min_data_in_leaf': (150, 250),
                                            'weight': (1, 300)}, random_state=1)
    # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round, acq="ei", kappa = 2.563)
    # El init_round realiza un randomGridSearch mientras que el opt_round realiza la optimizacion bayesiana
    
    # ucb para exploracion kappa=10 
    # ei para explotacion kappa=1 o xi=0 para exploracion xi=0.1
    
    # output optimization process
    if output_process==True: lgbBO.points_to_csv("bayes_opt_result.csv")
    
    # return best parameters
    return lgbBO.res

In [125]:
%%time
from bayes_opt import BayesianOptimization
opt_params = bayes_parameter_opt_lgb_auc(X_train,
                                         y_train, 
                                         init_round=20, 
                                         opt_round=5, 
                                         n_folds=40, 
                                         random_seed=1234, 
                                         n_estimators=500, 
                                         learning_rate=0.05)

|   iter    |  target   | baggin... | baggin... | featur... | lambda_l1 | lambda_l2 | max_depth | min_da... | min_ga... | min_su... | num_le... |  weight   |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
[200]	cv_agg's auc: 0.728994 + 0.00549352	cv_agg's gini: 0.457989 + 0.010987
|  1        |  0.729    |  0.5834   |  2.441    |  0.5      |  0.6744   |  0.3788   |  7.185    |  168.6    |  87.04    |  99.8     |  36.01    |  126.3    |
[200]	cv_agg's auc: 0.735899 + 0.00549318	cv_agg's gini: 0.471799 + 0.0109864
|  2        |  0.7359   |  0.637    |  1.409    |  0.6756   |  0.152    |  1.374    |  7.835    |  205.9    |  35.96    |  50.33    |  42.82    |  290.5    |
|  3        |  0.7199   |  0.5627   |  2.385    |  0.6753   |  1.8      |  0.2616   |  7.078    |  167.0    |  219.7    |  25.49    |  32.95    |  287.4    |
|  4        |  0.7196   |  0.6066   |  2.384    |  0.56

[400]	cv_agg's auc: 0.743002 + 0.00521843	cv_agg's gini: 0.486003 + 0.0104369
|  24       |  0.743    |  0.6115   |  1.447    |  0.6326   |  0.3464   |  1.379    |  8.272    |  249.7    |  10.67    |  246.0    |  45.7     |  297.4    |
[200]	cv_agg's auc: 0.745874 + 0.00510847	cv_agg's gini: 0.491749 + 0.0102169
[400]	cv_agg's auc: 0.747563 + 0.00503281	cv_agg's gini: 0.495126 + 0.0100656
|  25       |  0.7478   |  0.6219   |  1.257    |  0.5867   |  0.6279   |  1.668    |  7.122    |  160.8    |  3.46     |  197.0    |  47.42    |  270.8    |
CPU times: user 4h 59min 37s, sys: 32.1 s, total: 5h 9s
Wall time: 1h 41min 25s


In [126]:
AUCs = []
for i in range(25):
        AUCs.append(opt_params[i]['target'])
auc_max = max(AUCs)
best_params = opt_params[np.where(AUCs == auc_max)[0][0]]['params']
auc_max*2-1
best_params

0.49742913792600807

{'bagging_fraction': 0.5540557845037332,
 'bagging_freq': 1.5027016206117216,
 'feature_fraction': 0.5068213222676954,
 'lambda_l1': 1.0627672174901832,
 'lambda_l2': 0.6088814046649453,
 'max_depth': 7.407901504555802,
 'min_data_in_leaf': 239.0704612188054,
 'min_gain_to_split': 1.9569115749922075,
 'min_sum_hessian_in_leaf': 225.1642123539018,
 'num_leaves': 45.123194486569176,
 'weight': 234.38737874771672}